In [1]:
# 导入必要的库
import pandas as pd

# 读取 NVIDIA 股票数据
file_path = "/home/jesse/Projects/RL_Testing/SAC_Testing/CSVs/nvidia_stock_prices.csv"
df = pd.read_csv(file_path)
df.head()


,Date,Open,High,Low,Close,Volume
0,2004-01-02,0.196417,0.199083,0.192333,0.192333,436416000
1,2004-01-05,0.195250,0.199917,0.193500,0.198583,575292000
2,2004-01-06,0.198000,0.209417,0.197083,0.206667,1093344000
3,2004-01-07,0.204333,0.209500,0.202917,0.208500,673032000
4,2004-01-08,0.211083,0.212083,0.207250,0.209250,433752000


In [2]:
# 计算金融指标 （pandas-ta和ta-lib似乎都没法正常使用）

# 计算移动平均线（SMA & EMA）
for period in [5, 10, 25, 50, 100, 200, 500]:
    df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
    df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

# 计算 RSI（7天 & 14天）
def compute_rsi(data, window):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI_7'] = compute_rsi(df['Close'], 7)
df['RSI_14'] = compute_rsi(df['Close'], 14)

# 计算 MACD
short_ema = df['Close'].ewm(span=12, adjust=False).mean()
long_ema = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = short_ema - long_ema
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 计算布林带（Bollinger Bands, BB）
rolling_mean = df['Close'].rolling(window=20).mean()
rolling_std = df['Close'].rolling(window=20).std()
df['BB_upper'] = rolling_mean + (rolling_std * 2)
df['BB_lower'] = rolling_mean - (rolling_std * 2)

# 计算 ATR（平均真实波动幅度）
df['High-Low'] = df['High'] - df['Low']
df['High-Close'] = (df['High'] - df['Close'].shift(1)).abs()
df['Low-Close'] = (df['Low'] - df['Close'].shift(1)).abs()
df['True_Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
df['ATR_14'] = df['True_Range'].rolling(window=14).mean()

# 计算威廉指标（Williams %R）
low_min = df['Low'].rolling(window=14).min()
high_max = df['High'].rolling(window=14).max()
df['Williams_%R'] = (high_max - df['Close']) / (high_max - low_min) * -100

# 去掉NaN值
df = df.dropna()

# 检查数据
print(df.tail())

            Date       Open       High        Low      Close     Volume  \
5028  2023-12-22  49.195000  49.382999  48.466999  48.830002  252507000   
5029  2023-12-26  48.967999  49.599998  48.959999  49.278999  244200000   
5030  2023-12-27  49.511002  49.680000  49.084999  49.417000  233648000   
5031  2023-12-28  49.643002  49.883999  49.411999  49.521999  246587000   
5032  2023-12-29  49.813000  49.997002  48.750999  49.521999  389293000   

          SMA_5      EMA_5   SMA_10     EMA_10  ...      MACD  Signal_Line  \
5028  49.122401  48.821686  48.5224  48.531265  ...  0.633659     0.563715   
5029  48.962801  48.974124  48.7876  48.667217  ...  0.655050     0.581982   
5030  48.925401  49.121749  48.9636  48.803541  ...  0.675353     0.600656   
5031  49.207600  49.255166  49.1070  48.934170  ...  0.691939     0.618913   
5032  49.314000  49.344110  49.2242  49.041048  ...  0.697049     0.634540   

       BB_upper   BB_lower  High-Low  High-Close  Low-Close  True_Range  \
5028 

In [3]:
# 保存到新CSV文件
df.to_csv('/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/nvidia_stock_with_indicators.csv')

计算Apple公司股票的各项指标

In [4]:
# 1. 读取数据
file_path = "/home/jesse/Projects/RL_Testing/SAC_Testing/CSVs/apple_stock_prices.csv"
# df = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")
df = pd.read_csv(file_path)

# 2. 重命名列，并调整顺序
df = df.rename(columns={
    "Date" : "Date",
    "Open": "Open",
    "High": "High",
    "Low": "Low",
    "Close": "Close",
    "Volume": "Volume"
})
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]

df.head()

,Date,Open,High,Low,Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,73449600


In [5]:
# 计算金融指标 （pandas-ta和ta-lib似乎都没法正常使用）

# 计算移动平均线（SMA & EMA）
for period in [5, 10, 25, 50, 100, 200, 500]:
    df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
    df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

# 计算 RSI（7天 & 14天）
def compute_rsi(data, window):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI_7'] = compute_rsi(df['Close'], 7)
df['RSI_14'] = compute_rsi(df['Close'], 14)

# 计算 MACD
short_ema = df['Close'].ewm(span=12, adjust=False).mean()
long_ema = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = short_ema - long_ema
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 计算布林带（Bollinger Bands, BB）
rolling_mean = df['Close'].rolling(window=20).mean()
rolling_std = df['Close'].rolling(window=20).std()
df['BB_upper'] = rolling_mean + (rolling_std * 2)
df['BB_lower'] = rolling_mean - (rolling_std * 2)

# 计算 ATR（平均真实波动幅度）
df['High-Low'] = df['High'] - df['Low']
df['High-Close'] = (df['High'] - df['Close'].shift(1)).abs()
df['Low-Close'] = (df['Low'] - df['Close'].shift(1)).abs()
df['True_Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
df['ATR_14'] = df['True_Range'].rolling(window=14).mean()

# 计算威廉指标（Williams %R）
low_min = df['Low'].rolling(window=14).min()
high_max = df['High'].rolling(window=14).max()
df['Williams_%R'] = (high_max - df['Close']) / (high_max - low_min) * -100

# 去掉NaN值
df = df.dropna()

# 检查数据
print(df.tail())

             Date        Open        High         Low       Close    Volume  \
11102  2024-12-27  257.829987  258.700012  253.059998  255.589996  42355300   
11103  2024-12-30  252.229996  253.500000  250.750000  252.199997  35557500   
11104  2024-12-31  252.440002  253.279999  249.429993  250.419998  39480700   
11105  2025-01-02  248.929993  249.100006  241.820007  243.850006  55558000   
11106  2025-01-03  243.369995  244.179993  241.889999  243.860001  15135053   

            SMA_5       EMA_5      SMA_10      EMA_10  ...      MACD  \
11102  256.514001  255.832092  253.306000  253.378890  ...  6.126376   
11103  256.056000  254.621394  253.712999  253.164546  ...  5.659474   
11104  255.085999  253.220929  253.650999  252.665537  ...  5.087179   
11105  252.215997  250.097288  252.688000  251.062714  ...  4.056724   
11106  249.184000  248.018192  252.269000  249.753129  ...  3.203954   

       Signal_Line    BB_upper    BB_lower  High-Low  High-Close  Low-Close  \
11102     5.6

In [6]:
# 保存到新CSV文件
df.to_csv('/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/apple_stock_with_indicators.csv')

计算微软股票指标

In [7]:
# 1. 读取数据
file_path = "/home/jesse/Projects/RL_Testing/SAC_Testing/CSVs/microsoft_stock_price.csv"
# df = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")
df = pd.read_csv(file_path)

# 2. 重命名列，并调整顺序
df = df.rename(columns={
    "Date" : "Date",
    "Open": "Open",
    "High": "High",
    "Low": "Low",
    "Close": "Close",
    "Volume": "Volume"
})
df = df[["Date", "Open", "High", "Low", "Close", "Volume"]]

df.head()

,Date,Open,High,Low,Close,Volume
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,1031788800
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,308160000
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,133171200
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,67766400
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,47894400


In [8]:
# 计算金融指标 （pandas-ta和ta-lib似乎都没法正常使用）

# 计算移动平均线（SMA & EMA）
for period in [5, 10, 25, 50, 100, 200, 500]:
    df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
    df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

# 计算 RSI（7天 & 14天）
def compute_rsi(data, window):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI_7'] = compute_rsi(df['Close'], 7)
df['RSI_14'] = compute_rsi(df['Close'], 14)

# 计算 MACD
short_ema = df['Close'].ewm(span=12, adjust=False).mean()
long_ema = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = short_ema - long_ema
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 计算布林带（Bollinger Bands, BB）
rolling_mean = df['Close'].rolling(window=20).mean()
rolling_std = df['Close'].rolling(window=20).std()
df['BB_upper'] = rolling_mean + (rolling_std * 2)
df['BB_lower'] = rolling_mean - (rolling_std * 2)

# 计算 ATR（平均真实波动幅度）
df['High-Low'] = df['High'] - df['Low']
df['High-Close'] = (df['High'] - df['Close'].shift(1)).abs()
df['Low-Close'] = (df['Low'] - df['Close'].shift(1)).abs()
df['True_Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
df['ATR_14'] = df['True_Range'].rolling(window=14).mean()

# 计算威廉指标（Williams %R）
low_min = df['Low'].rolling(window=14).min()
high_max = df['High'].rolling(window=14).max()
df['Williams_%R'] = (high_max - df['Close']) / (high_max - low_min) * -100

# 去掉NaN值
df = df.dropna()

# 检查数据
print(df.tail())

            Date        Open        High         Low       Close    Volume  \
9078  2022-03-18  295.369995  301.000000  292.730011  300.429993  43317000   
9079  2022-03-21  298.890015  300.140015  294.899994  299.160004  28351200   
9080  2022-03-22  299.799988  305.000000  298.769989  304.059998  27599700   
9081  2022-03-23  300.510010  303.230011  297.720001  299.489990  25715400   
9082  2022-03-24  299.140015  304.200012  298.320007  304.100006  24446900   

           SMA_5       EMA_5      SMA_10      EMA_10  ...      MACD  \
9078  290.726001  293.310819  286.255002  290.635141  ... -2.326018   
9079  295.270001  295.260548  288.280002  292.185116  ... -1.458543   
9080  298.652002  298.193698  291.101001  294.344186  ... -0.371393   
9081  299.671997  298.625795  292.200000  295.279787  ...  0.120036   
9082  301.447998  300.450532  294.051001  296.883463  ...  0.871442   

      Signal_Line    BB_upper    BB_lower  High-Low  High-Close  Low-Close  \
9078    -4.321428  305.432

In [9]:
# 保存到新CSV文件
df.to_csv('/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/microsoft_stock_with_indicators.csv')